In [1]:
from pytrends.request import TrendReq
import pandas as pd
import datetime
from datetime import date
import numpy as np
import os
import time
import re

In [2]:
#要改搜尋年月日
#設定Google Trends區間
set_begin_date = datetime.datetime(2016, 1, 1)
set_end_date = datetime.datetime(2019, 12, 31)

print(set_begin_date.date() , set_end_date.date())
user_timeframe = set_begin_date.strftime('%Y-%m-%d')+' '+ set_end_date.strftime('%Y-%m-%d')

2016-01-01 2019-12-31


In [3]:
#要輸入查詢的keyword
pytrend = TrendReq()
plist = []
trends_dict = ['kobe','jordan','curry','zion','lebron','warriors','lakers','bucks','iverson','nba','embiid','derrick'
              ,'kawhi','giannis']
for keywords in trends_dict:
    print(keywords)
    keywords_list = [keywords]
    
    pytrend.build_payload(kw_list=keywords_list, cat=0, timeframe=user_timeframe, geo='TW')
    interest_over_time_df = pytrend.interest_over_time()
    count_row = interest_over_time_df.shape[0]
    plist.append(keywords)
    
    for date in range(count_row):
        if len(interest_over_time_df.iloc[:])==0:
            plist.append(0)
        else:
            plist.append(interest_over_time_df.iloc[date,0])

kobe
jordan
curry
zion
lebron
warriors
lakers
bucks
iverson
nba
embiid
derrick
kawhi
giannis


In [4]:
#將index轉column 合併在原本dataframe
interest_over_time_df['date'] = interest_over_time_df.index

In [5]:
#新增最後一行為date 取最後一行
interest_over_time_df = interest_over_time_df.iloc[:,-1]

In [6]:
#刪掉Index欄位
interest_over_time_df.reset_index(drop=True, inplace=True)

In [7]:
#將搜尋結果變成str
plist_str = [str(p) for p in plist]

In [8]:
trends_num = []

In [9]:
#單純取出數字行欄位
for p in plist_str:
    if len(re.findall('[0-9]+',p))>0:
        trends_num.append(p)

In [10]:
#分組
trends_num_arr = np.array(trends_num,dtype = int)
arr_split = np.split(trends_num_arr,len(trends_dict))

In [11]:
myarray = np.asarray(arr_split)

In [12]:
#轉型態
df = pd.DataFrame(myarray)

In [13]:
#Row變column column變row
df_transposed = df.T

In [14]:
#Column命名
df_transposed.columns = trends_dict

In [15]:
#合併date跟搜尋結果
new_df = pd.concat([interest_over_time_df,df_transposed], axis=1)

In [16]:
new_df

,date,kobe,jordan,curry,zion,lebron,warriors,lakers,bucks,iverson,nba,embiid,derrick,kawhi,giannis
0,2016-01-03,12,71,36,21,19,24,41,6,45,25,0,14,5,0
1,2016-01-10,14,74,36,10,21,30,20,8,23,25,0,18,5,10
2,2016-01-17,12,60,37,8,21,32,22,12,17,24,0,11,7,19
3,2016-01-24,11,58,42,10,18,30,18,6,28,28,0,12,7,0
4,2016-01-31,13,78,35,15,18,24,16,24,28,26,0,13,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2019-12-01,4,41,17,25,14,20,76,49,10,42,28,8,4,21
205,2019-12-08,3,42,17,19,13,12,72,36,8,44,23,13,8,32
206,2019-12-15,4,42,19,15,19,18,83,89,8,44,15,12,6,53
207,2019-12-22,4,42,20,21,15,24,78,38,9,41,22,11,7,40


In [17]:
#做每月平均
#因為要做每個月平均 時間獨立出來
unique_date_list = new_df['date'].dt.strftime("%m/%Y").unique().tolist()

In [18]:
#將獨立出來的時間欄位命名
unique_date_dataframe = pd.DataFrame(unique_date_list,columns = ['Date']) 

In [19]:
#把年份獨立出來
year_list = new_df['date'].dt.strftime("%Y").unique().tolist()

In [20]:
#把list內部的string轉成int
for i in range(0, len(year_list)): 
    year_list[i] = int(year_list[i]) 

In [21]:
avg_month = []

In [22]:
#每個月做平均
year = year_list
for j in year:
    df = new_df.loc[new_df["date"].dt.year == j,]
    for i in range(0,12):
        avg_month.append(df.loc[df["date"].dt.month == i+1,].mean())

In [23]:
#做完平均後轉DataFrame
avg_month = pd.DataFrame(avg_month, columns =trends_dict)

In [24]:
#合併Unique的Date 跟搜尋熱度的每月平均
result = pd.concat([unique_date_dataframe,avg_month], axis=1)

In [25]:
result

,Date,kobe,jordan,curry,zion,lebron,warriors,lakers,bucks,iverson,nba,embiid,derrick,kawhi,giannis
0,01/2016,12.40,68.20,37.20,12.80,19.40,28.00,23.40,11.20,28.20,25.60,0.00,13.60,6.00,5.80
1,02/2016,13.25,80.50,60.50,8.75,19.00,24.00,14.75,8.00,36.75,25.25,2.50,11.00,4.00,6.75
2,03/2016,11.25,53.75,47.00,9.50,15.75,25.25,15.00,9.25,30.50,26.75,14.75,12.00,6.75,14.50
3,04/2016,38.25,59.00,58.25,8.50,16.50,29.75,23.75,8.75,58.00,36.50,4.75,11.50,7.50,10.50
4,05/2016,10.60,44.00,54.00,8.60,19.40,48.00,14.20,8.40,26.80,46.00,4.60,11.00,4.40,5.40
5,06/2016,13.00,44.25,65.75,13.00,52.50,56.25,18.75,6.50,40.75,40.25,8.25,25.25,2.75,7.50
6,07/2016,8.20,37.40,25.60,9.80,16.20,16.00,19.20,10.80,20.60,11.80,13.40,10.60,2.20,11.20
7,08/2016,5.75,41.25,21.00,8.75,9.75,5.75,9.25,9.75,16.75,6.00,11.25,7.00,1.75,2.25
8,09/2016,7.25,41.25,32.75,8.50,11.25,6.50,10.25,8.00,47.50,6.50,8.50,7.25,1.50,8.75
9,10/2016,6.20,33.20,22.80,10.40,11.60,15.40,19.20,9.00,17.80,18.60,25.60,12.80,4.20,8.40


In [26]:
result.to_csv("result.csv", index=True)